In [ ]:
from data_loader import SpectLoader
import torch
from torch import nn, optim
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch.nn import ReLU
from torch.optim.lr_scheduler import ReduceLROnPlateau
from basic_model import BasicModel
from skip_network import SkipNetwork
from model import ConvNetwork
from config import hdf5_path, spec_minmax_scaler_path, spec_log_transformer_path


In [ ]:
def data_actual_dist(pprocessor, train_keys, num_bins=50):
    bin_edges = np.linspace(start=-1.0, stop=1.0, num=num_bins + 1)  # Adjust the range based on your data's expected value range

    bin_counts = np.zeros(num_bins)
    for x_batch, _ in pprocessor.batch_generator(train_keys):
        resized_tensor = F.interpolate(x_batch, size=(128, 512), mode='bilinear', align_corners=False, antialias=True)
        
        data = resized_tensor.numpy() if hasattr(resized_tensor, 'numpy') else resized_tensor

        flattened_data = data.flatten()

        hist, _ = np.histogram(flattened_data, bins=bin_edges)
        bin_counts += hist  

    # Normalize the bin counts to get probabilities (optional)
    bin_counts_normalized = bin_counts / bin_counts.sum()

    plt.figure(figsize=(10, 6))
    plt.bar(
        bin_edges[:-1], 
        bin_counts_normalized, 
        width=np.diff(bin_edges), 
        align="edge", 
        color="blue", 
        alpha=0.7
    )
    plt.title("Value Distribution in x_batch")
    plt.xlabel("Value")
    plt.ylabel("Frequency (Normalized)")
    plt.grid(True)
    plt.show()

    return bin_edges

def plot_spect(sepct):
    plt.figure(figsize=(10, 5))  # Adjust figure size as needed
    plt.imshow(sepct, aspect='auto', cmap='hot', origin='lower')
    plt.colorbar(label='Intensity')  # Optional: Add a colorbar
    plt.title('Spectrogram Heatmap')
    plt.xlabel('Time')
    plt.ylabel('Frequency')
    plt.show()

def plot_several(batch, n_show):
    for i in range(n_show):
        matrix = batch[i].squeeze(0).numpy()
        plot_spect(matrix)




In [ ]:
paths = {
        "data_path": hdf5_path,
        "scaler_path": spec_minmax_scaler_path
    }

pprocessor = SpectLoader(paths, batch_size=32)
train_keys, val_keys, test_keys = pprocessor.split_data()
pprocessor.setup_pipeline(scaler_type="normalizer",load_model=True)

In [ ]:
sample1_shape = pprocessor.spect_data[pprocessor.train_keys[0]]['spectrogram'].shape
input_shape=(1, sample1_shape[0], sample1_shape[1])
print(input_shape)

In [ ]:
model = BasicModel(input_shape=input_shape)
total_params = sum(p.numel() for p in model.parameters())
print(total_params)

In [ ]:
model = SkipNetwork(channels=[128,128,128,128], input_shape=input_shape)
total_params = sum(p.numel() for p in model.parameters())
print(total_params)

3148424


In [ ]:
#model = ConvNetwork(input_shape=input_shape)
model = SkipNetwork(channels=[128,128,128,128], input_shape=input_shape)
#model = BasicModel(input_shape=input_shape)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

batch_size = 16

criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.7, patience=1)
print(f"1: Used CUDA memory: {torch.cuda.memory_allocated() / 1e6} MB")

for i in range(20):
    train_accs = []
    train_losses = []

    val_accs = []
    val_losses = []
    for x, y in pprocessor.batch_generator(train_keys, batch_size=batch_size):
        optimizer.zero_grad()    

        outputs = model(x)
        loss = criterion(outputs, y)

        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            _, preds = torch.max(outputs, 1)
            train_accuracy = (preds == y).float().mean().item()

        train_accs.append(train_accuracy)
        train_losses.append(loss.item())

        
    model.eval()
    with torch.no_grad():
        for x, y in pprocessor.batch_generator(val_keys, batch_size=batch_size):
            outputs = model(x)
            val_loss = criterion(outputs, y).item()
        
            _, preds = torch.max(outputs, 1)
            val_accuracy = (preds == y).float().mean().item()
            
            val_accs.append(val_accuracy)
            val_losses.append(val_loss)
        
        scheduler.step(np.mean(val_losses))
        print(f"3: Used CUDA memory: {torch.cuda.memory_allocated() / 1e6} MB")


    
    print(f'Avg. Training Loss: {np.mean(train_losses):.4f}, Avg. Train Accuracy: {np.mean(train_accs):.4f}, Val Loss: {np.mean(val_losses):.4f}, Val Accuracy: {np.mean(val_accs):.4f},')